# Morris sampling sensitivity analysis for Forrest local SDGs systems model

In [2]:
import sys
# set local file paths
sys.path.append(r'C:\temp')
sys.path.append(r'C:\Users\')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench import load_results, ema_logging
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import morris
import os
from datetime import datetime

C:\Users\szeteyka\Anaconda3\lib\site-packages\ema_workbench\em_framework\evaluators.py:21: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn(
C:\Users\szeteyka\Anaconda3\lib\site-packages\ema_workbench\em_framework\optimization.py:48: ImportWarning: platypus based optimization not available
  warnings.warn("platypus based optimization not available", ImportWarning)


## Load model, uncertainities, outcomes; Run experiments

### before running this code, either rename the save file or delete the previous version

In [3]:
def generate_experiments(n_scenarios):
    
    #The model must be imported as .py file in parallel processing.
    from ema_workbench.connectors.vensim import VensimModel
     
    #set local file paths
    
    directory = 'C:/temp/scenarios/'
    vensimModel = VensimModel("LSP6", wd=directory, model_file=r'C:/temp/scenarios/scenariomodel.vpmx') #change here
    
    from ema_workbench import (TimeSeriesOutcome, 
                                   perform_experiments,
                                   RealParameter, 
                                   CategoricalParameter,
                                   Constant,
                                   ema_logging, 
                                   save_results,
                                  load_results)

    directory = 'C:/temp/scenarios/' #change here

    df_unc = pd.read_excel(directory+'scenario_params.xlsx', sheet_name='scenario_params')
    
    #Specify uncertainties
    vensimModel.uncertainties = [RealParameter(row['Variables'], row['LSP6Lower'], row['LSP6Upper']) for index, row in df_unc.iterrows()] #change here
    
    #Specify outcomes
    df_out = pd.read_excel(directory+'scenario_params.xlsx', sheet_name='outcomes')
    vensimModel.outcomes = [TimeSeriesOutcome(out) for out in df_out['Outcome']]

    from ema_workbench import MultiprocessingEvaluator
    from ema_workbench.em_framework.evaluators import (MC, LHS, FAST, FF, PFF, SOBOL, MORRIS)


    try: 
        with MultiprocessingEvaluator(vensimModel, n_processes=3) as evaluator:
            results = evaluator.perform_experiments(scenarios=n_scenarios, uncertainty_sampling=MORRIS)
    except (BrokenPipeError, IOError):
        pass

    fn = 'C:/temp/scenarios/LSP6/LSP6{}.tar.gz'.format(n_scenarios) #change here

    # If scenario results file exists, move to backup (replacing the backup if it exists)
    
    if os.path.isfile(fn):
        os.replace(fn, fn + '.bak')
    
    save_results(results, fn)
    
    experiments, outcomes = results
    
    return vensimModel.uncertainties, experiments, outcomes, results

In [4]:
# If Morris index .csv files already exist in the save directory, you MUST delete them as this code does not overwrite them

def make_morris_df(scores, problem, outcome_var, sc, t):
    # print("make_morris_df {}".format(outcome_var))
    scores_filtered = {k:scores[k] for k in ['mu_star','mu_star_conf','mu','sigma']}
    Si_df = pd.DataFrame(scores_filtered, index=problem['names'])
    sa_dir = 'C:/temp/scenarios/LSP6/' #change here
    Si_df.to_csv(sa_dir+"MorrisIndices_{}_sc{}_t{}_test.csv".format(outcome_var, sc, t))
    
    Si_df.sort_values(by=['mu_star'], ascending=False, inplace=True)
    Si_df = Si_df.head(20)
    
    Si_df = Si_df.iloc[::-1]

    indices = Si_df[['mu_star','mu']]
    errors = Si_df[['mu_star_conf','sigma']]
    return indices, errors

In [5]:
def plot_scores(inds, errs, outcome_var, sc, t):
    #print("plot_scores {}".format(outcome_var))
    sns.set_style('white')
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3, 6)) 
    ind = inds[outcome_var].iloc[:,0]
    err = errs[outcome_var].iloc[:,0]
    ind.plot.barh(xerr=err.values.T,ax=ax, color = ['#62C890'], width=.9)
    ax.set_ylabel('')
    ax.legend().set_visible(False)
    ax.set_xlabel('mu_star index', fontsize=14)

    ylabels = ax.get_yticklabels()
#    ylabels = [item.get_text()[:-10] for item in ylabels]
    ax.set_yticklabels(ylabels, fontsize=10)
    #ax.set_title("2100", fontsize=12)

    # datetime object containing current date and time
    now = datetime.now()
 
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%Y%M%d-%H%M")
    #print("date and time =", dt_string)

    plt.suptitle('{} in {}'.format(outcome_var, t), y=0.94, fontsize=12)
    plt.rcParams["figure.figsize"] = [7.08,7.3]
    plt.savefig('{}{}-Morris_ranking_{}_sc{}_{}.png'.format(r'C:/temp/scenarios/LSP6/', dt_string, outcome_var, sc, t),  #change here
                dpi=600,  bbox_inches='tight')
    return fig

In [6]:
if __name__ == '__main__':
    
    ema_logging.log_to_stderr(ema_logging.INFO)

    targetyear_ind = {2030: 30}
    import  time
    start = time.time()
    for n_scenarios in [2000]:
        #print("Starting {} scenarios".format(n_scenarios))
        uncertainties, experiments, outcomes, results = generate_experiments(n_scenarios)
        sc = n_scenarios
        outcome_vars = list(outcomes.keys())[1:]
        problem = get_SALib_problem(uncertainties)
        for t in list(targetyear_ind.keys()):
            #print("Year {}".format(t))
            inds ={}
            errs = {}
            for i, outcome_var in enumerate(outcome_vars):
                # print("i {} outcome_var {}".format(i,outcome_var))
                X = experiments.iloc[:, :-3].values
                Y = outcomes[outcome_var][:,targetyear_ind[t]]
                scores = morris.analyze(problem, X, Y, print_to_console=False)
                inds[outcome_var], errs[outcome_var] = make_morris_df(scores, problem, outcome_var, sc, t)
            for out, outcome_var in enumerate(outcome_vars):
                plot_scores(inds, errs, outcome_var, sc, t)
                plt.close()
    end = time.time()
    print("took {} seconds".format(end-start))

[MainProcess/INFO] using 64 bit vensim
C:\Users\szeteyka\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:27: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 76000 scenarios * 1 policies * 1 model(s) = 76000 experiments
[MainProcess/INFO] 7600 cases completed
[MainProcess/INFO] 15200 cases completed
[MainProcess/INFO] 22800 cases completed
[MainProcess/INFO] 30400 cases completed
[MainProcess/INFO] 38000 cases completed
[MainProcess/INFO] 45600 cases completed
[MainProcess/INFO] 53200 cases completed
[MainProcess/INFO] 60800 cases completed
[MainProcess/INFO] 68400 cases completed
[MainProcess/INFO] 76000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] results saved successfully to C:\temp\scenarios\LSP6\LSP62000.tar.gz


took 28618.376435041428 seconds


In [ ]:
    # Run just the plots
    
    start = time.time()
    for n_scenarios in [2000]:
        #print("Starting {} scenarios".format(n_scenarios))
        #uncertainties, experiments, outcomes, results = generate_experiments(n_scenarios)
        sc = n_scenarios
        outcome_vars = list(outcomes.keys())[1:]
        problem = get_SALib_problem(uncertainties)
        for t in list(targetyear_ind.keys()):
            #print("Year {}".format(t))
            inds ={}
            errs = {}
            for i, outcome_var in enumerate(outcome_vars):
                # print("i {} outcome_var {}".format(i,outcome_var))
                X = experiments.iloc[:, :-3].values
                Y = outcomes[outcome_var][:,targetyear_ind[t]]
                scores = morris.analyze(problem, X, Y, print_to_console=False)
                inds[outcome_var], errs[outcome_var] = make_morris_df(scores, problem, outcome_var, sc, t)
            for out, outcome_var in enumerate(outcome_vars):
                plot_scores(inds, errs, outcome_var, sc, t)
                plt.close()
    end = time.time()
    print("took {} seconds".format(end-start))

In [8]:
#save_results(results, fn)
    
experiments, outcomes = results
    
#return vensimModel.uncertainties, experiments, outcomes, results

In [18]:
results = load_results('C:/temp/scenarios/LSP2/LSP22000.tar.gz') #change here
experiments, outcomes = results
dict_outcomes = outcomes

In [19]:
outcomes_df = pd.DataFrame(columns=list(dict_outcomes.keys()))
for key in set(list(dict_outcomes.keys())): 
    outcomes_df[key] = dict_outcomes[key][:, 0:].ravel() # Specify from when to plot the data 
outcomes_dict = outcomes_df

outcomes_df

,TIME,total population,Housing Land,housing demand,Total Other Productivity,Species Richness,inequality indicator,Safer Healthier People,Internet Service Demand,whole population bus frequency discrepancy
0,2000.00,171.00000,57.300000,85.500000,1175000.0,4080.5334,0.125612,171.00000,141.00000,197.08331
1,2000.25,171.43889,83.472190,85.719444,1209787.6,4074.8200,0.124553,170.88370,141.04994,200.52530
2,2000.50,171.86124,101.338380,85.969140,1262482.2,4069.2010,0.125247,170.76825,141.10341,203.83765
3,2000.75,172.24217,115.997635,86.236660,1324147.9,4063.6472,0.123036,170.65366,141.16106,206.82516
4,2001.00,172.60416,128.763820,86.533220,1394855.6,4058.1526,0.122762,170.53989,141.22223,209.66402
...,...,...,...,...,...,...,...,...,...,...
15275995,2049.00,657.86140,716.760500,434.034330,12343758.0,4296.9907,0.204613,162.24051,163.77266,3807.33860
15275996,2049.25,664.66693,718.134340,438.278320,12459784.0,4297.5283,0.204678,162.24632,164.00473,3860.71200
15275997,2049.50,671.57710,719.501160,442.574500,12576693.0,4298.0860,0.204742,162.25215,164.23938,3914.90530
15275998,2049.75,678.57620,720.860900,446.914980,12694285.0,4298.6636,0.204809,162.25800,164.47606,3969.79610


In [ ]:
# datetime object containing current date and time
now = datetime.now()
 
# dd/mm/YY H:M:S
dt_string = now.strftime("%Y%m%d-%H%M")
#print("date and time =", dt_string)

outcomes_df.to_csv('C:/temp/scenarios/LSP2/LSP2_data.csv'.format(dt_string)) #change here